In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Clase para cargar y preparar los datos
class LoadData:
    def __init__(self, file_path):
        self.file_path = file_path

    def load_data(self):
        data = pd.read_csv(self.file_path)
        return data

    def prepare_data(self, data):
        # Eliminar columnas irrelevantes
        data = data.drop(['idPartido', 'Ronda', 'Temporada', 'Evento'], axis=1)

        # Separar características y etiquetas
        X = data.drop(['VictoriaLocal', 'Empate', 'VictoriaVisitante', 'GolesLocal', 'GolesVisitante'], axis=1)
        y = data[['GolesLocal', 'GolesVisitante']]  # Seleccionar goles locales y visitantes como etiquetas

        # Escalar características
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # Dividir los datos en conjuntos de entrenamiento y prueba
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
        
        # Convertir los datos a tensores
        X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
        X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
        y_train = tf.convert_to_tensor(y_train.values, dtype=tf.float32)
        y_test = tf.convert_to_tensor(y_test.values, dtype=tf.float32)
        
        return X_train, X_test, y_train, y_test, scaler, X, y

# Cargar los datos
data_loader = LoadData('dataframe/champions.csv')
data = data_loader.load_data()
X_train, X_test, y_train, y_test, scaler, X, y = data_loader.prepare_data(data)


In [13]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Reshape, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop


# Clase para entrenar y evaluar el modelo de predicción de goles locales y visitantes
class GoalsPredictionModel:
    def __init__(self, configurations):
        self.configurations = configurations
        self.best_model = None
        self.best_config = None
        self.best_mae = float('inf')

    def train_model(self, X_train, y_train, X_test, y_test):
        for config in self.configurations:
            input_layer = Input(shape=(X_train.shape[1],))
            reshaped_input = Reshape((X_train.shape[1], 1))(input_layer)
            conv_layer = Conv1D(config['filters'], config['kernel_size'], activation='relu')(reshaped_input)
            pooled_layer = MaxPooling1D()(conv_layer)
            flattened_layer = Flatten()(pooled_layer)
            dense_layer1 = Dense(config['units'], activation='relu')(flattened_layer)
            dropout_layer = Dropout(config['dropout'])(dense_layer1)
            
            # Salida para la predicción de goles locales
            local_goals_output = Dense(1, name='local_goals_output')(dropout_layer)
            # Salida para la predicción de goles visitantes
            visitor_goals_output = Dense(1, name='visitor_goals_output')(dropout_layer)
            
            model = Model(inputs=input_layer, outputs=[local_goals_output, visitor_goals_output])

            optimizer = Adam(learning_rate=config['learning_rate'])
            model.compile(optimizer=optimizer, loss='mse', metrics=['mae', 'mae'])
            
            history = model.fit(X_train, [y_train[:, 0], y_train[:, 1]], epochs=config['epochs'], batch_size=config['batch_size'], validation_split=0.1)
            
            _, local_mae, visitor_mae = model.evaluate(X_test, [y_test[:, 0], y_test[:, 1]])
            total_mae = (local_mae + visitor_mae) / 2
            
            if total_mae < self.best_mae:
                self.best_mae = total_mae
                self.best_model = model
                self.best_config = config
    
    def get_best_model(self):
        return self.best_model
    
    def get_best_config(self):
        return self.best_config

# Definir diferentes configuraciones de red y hiperparámetros
configurations = [
    {'units': 64, 'filters': 32, 'kernel_size': 3, 'learning_rate': 0.001, 'batch_size': 32, 'epochs': 10, 'dropout': 0.2},
    {'units': 128, 'filters': 64, 'kernel_size': 3, 'learning_rate': 0.001, 'batch_size': 64, 'epochs': 15, 'dropout': 0.1},
    {'units': 256, 'filters': 128, 'kernel_size': 5, 'learning_rate': 0.0001, 'batch_size': 16, 'epochs': 10, 'dropout': 0.3},
    {'units': 64, 'filters': 32, 'kernel_size': 3, 'learning_rate': 0.001, 'batch_size': 64, 'epochs': 20, 'dropout': 0.3},
    {'units': 128, 'filters': 64, 'kernel_size': 5, 'learning_rate': 0.001, 'batch_size': 32, 'epochs': 15, 'dropout': 0.2},
    {'units': 256, 'filters': 128, 'kernel_size': 3, 'learning_rate': 0.0005, 'batch_size': 32, 'epochs': 10, 'dropout': 0.1},
    {'units': 64, 'filters': 32, 'kernel_size': 5, 'learning_rate': 0.001, 'batch_size': 64, 'epochs': 10, 'dropout': 0.1},
    {'units': 128, 'filters': 64, 'kernel_size': 3, 'learning_rate': 0.001, 'batch_size': 32, 'epochs': 20, 'dropout': 0.2},
    {'units': 256, 'filters': 128, 'kernel_size': 5, 'learning_rate': 0.0005, 'batch_size': 32, 'epochs': 15, 'dropout': 0.2},
    {'units': 128, 'filters': 64, 'kernel_size': 3, 'learning_rate': 0.01, 'batch_size': 64, 'epochs': 15, 'dropout': 0.1},
    {'units': 256, 'filters': 128, 'kernel_size': 5, 'learning_rate': 0.0001, 'batch_size': 16, 'epochs': 10, 'dropout': 0.3},
    {'units': 64, 'filters': 32, 'kernel_size': 3, 'learning_rate': 0.001, 'batch_size': 64, 'epochs': 20, 'dropout': 0.3},
    {'units': 128, 'filters': 64, 'kernel_size': 5, 'learning_rate': 0.001, 'batch_size': 32, 'epochs': 10, 'dropout': 0.2},
    {'units': 256, 'filters': 128, 'kernel_size': 3, 'learning_rate': 0.001, 'batch_size': 16, 'epochs': 15, 'dropout': 0.1},
    {'units': 64, 'filters': 32, 'kernel_size': 3, 'learning_rate': 0.01, 'batch_size': 64, 'epochs': 10, 'dropout': 0.1},
    {'units': 128, 'filters': 64, 'kernel_size': 5, 'learning_rate': 0.001, 'batch_size': 32, 'epochs': 10, 'dropout': 0.1},
    {'units': 64, 'filters': 32, 'kernel_size': 3, 'learning_rate': 0.001, 'batch_size': 32, 'epochs': 10, 'dropout': 0.2},
    {'units': 128, 'filters': 64, 'kernel_size': 3, 'learning_rate': 0.01, 'batch_size': 64, 'epochs': 15, 'dropout': 0.1},
    {'units': 256, 'filters': 128, 'kernel_size': 5, 'learning_rate': 0.0001, 'batch_size': 16, 'epochs': 10, 'dropout': 0.3},
    {'units': 64, 'filters': 32, 'kernel_size': 3, 'learning_rate': 0.001, 'batch_size': 64, 'epochs': 20, 'dropout': 0.3}
]


goals_model_trainer = GoalsPredictionModel(configurations)
goals_model_trainer.train_model(X_train, y_train, X_test, y_test)

goals_model = goals_model_trainer.get_best_model()
best_config = goals_model_trainer.get_best_config()
print("Mejor configuración:", best_config)
    


Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - local_goals_output_mae: 1.2854 - loss: 4.7443 - visitor_goals_output_mae: 1.0071 - val_local_goals_output_mae: 1.0042 - val_loss: 2.9144 - val_visitor_goals_output_mae: 0.9567
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - local_goals_output_mae: 1.0781 - loss: 3.4319 - visitor_goals_output_mae: 0.9608 - val_local_goals_output_mae: 1.0048 - val_loss: 2.7011 - val_visitor_goals_output_mae: 0.8523
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 971us/step - local_goals_output_mae: 1.0238 - loss: 3.1072 - visitor_goals_output_mae: 0.8932 - val_local_goals_output_mae: 0.9983 - val_loss: 2.6823 - val_visitor_goals_output_mae: 0.8631
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 972us/step - local_goals_output_mae: 1.0327 - loss: 3.1356 - visitor_goals_output_mae: 0.8571 - val_local_goals_output_mae: 1.0161 - val_loss: 2.6559 - val_visitor_goals_output_mae: 0.8477
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 984us/step - local_goals_output_mae: 1.0074

In [14]:
from sklearn.metrics import mean_absolute_error

class ModelEvaluation:
    def __init__(self, model):
        self.model = model
    
    def evaluate_model(self, X_test, y_test):
        # Evaluar el modelo
        local_goals_pred, visitor_goals_pred = self.model.predict(X_test)

        # Calcular y mostrar el MAE para cada tipo de predicción
        local_mae = mean_absolute_error(y_test[:, 0], local_goals_pred)
        visitor_mae = mean_absolute_error(y_test[:, 1], visitor_goals_pred)
        
        print("MAE for local goals prediction:", local_mae)
        print("MAE for visitor goals prediction:", visitor_mae)

model_evaluator = ModelEvaluation(goals_model)
model_evaluator.evaluate_model(X_test, y_test)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
MAE for local goals prediction: 1.0201901
MAE for visitor goals prediction: 0.86689365
